In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests
import time

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 2,
    "ProductOffering": 3,
    "Facility": 4,
    "Parts": 5,
    "Warehouse": 6,
    "Supplier": 7,
}

In [3]:
# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000"

version = "test1"

In [4]:
create_ops, graph = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

from networkx.readwrite import json_graph
print(json_graph.node_link_data(graph[0], edges="links"))

direct_create_payload = {
    "version": version,
    "action": "direct_create",
    "type": "schema",
    "timestamp": create_ops[0]["timestamp"],
    "payload": json_graph.node_link_data(graph[0], edges="links")
}

requests.post(f"{API_URL}/schema/live/update", json=direct_create_payload)


Generated 683 operations
{'directed': True, 'multigraph': False, 'graph': {}, 'nodes': [{'id': '1', 'name': 'Name_3542', 'description': 'Description_2030', 'type': 'Type C', 'cost': 2768.05, 'importance': 4, 'expected_life': 2475, 'units_in_chain': 37, 'expiry': 128, 'node_type': 'Parts', 'usage': 'core'}, {'id': '2', 'name': 'Name_1961', 'description': 'Description_8365', 'type': 'Type C', 'cost': 6056.73, 'importance': 1, 'expected_life': 1173, 'units_in_chain': 72, 'expiry': 324, 'node_type': 'Parts', 'usage': 'core'}, {'id': '3', 'name': 'Name_5813', 'description': 'Description_8897', 'type': 'Type C', 'cost': 8436.05, 'importance': 1, 'expected_life': 2583, 'units_in_chain': 6, 'expiry': 120, 'node_type': 'Parts', 'usage': 'core'}, {'id': '4', 'name': 'Name_8359', 'description': 'Description_2835', 'type': 'Type A', 'cost': 7955.66, 'importance': 1, 'expected_life': 2413, 'units_in_chain': 6, 'expiry': 56, 'node_type': 'Parts', 'usage': 'core'}, {'id': '5', 'name': 'Name_4252', 'd

<Response [200]>

In [5]:
# bulk_create_payload = {
#     "version": version,
#     "action": "bulk_create",
#     "type": "schema",
#     "timestamp": create_ops[0]["timestamp"],
#     "payload": []
# }

# for op in create_ops:
#     bulk_create_payload["payload"].append(op["payload"])

# print(bulk_create_payload)

# requests.post(f"{API_URL}/schema/live/update", json=bulk_create_payload)

In [6]:
update_cycles = 5
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    bulk_update_payload = {
        "version": version,
        "action": "bulk_update",
        "type": "schema",
        "timestamp": update_ops[0]["timestamp"],
        "payload": []
    }

    for op in update_ops:
        bulk_update_payload["payload"].append(op["payload"])

    print(bulk_update_payload)

    requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
    time.sleep(1)

Generated 19 update operations
{'version': 'test1', 'action': 'bulk_update', 'type': 'schema', 'timestamp': 0, 'payload': [{'node_id': '1-3', 'node_type': 'Facility', 'payload': {'id': '1-3', 'name': 'Name_3025', 'type': 'Type A', 'location': 'Phoenix', 'max_capacity': 4050, 'operating_cost': 9050.19}}, {'node_id': 'Supplier_6', 'node_type': 'Supplier', 'payload': {'id': 'Supplier_6', 'name': 'Name_8119', 'location': 'Los Angeles', 'quality': 685.28, 'reliability': 0.15, 'size': 'Small'}}, {'node_id': 'Warehouse_6', 'node_type': 'Warehouse', 'payload': {'id': 'Warehouse_6', 'name': 'Name_6968', 'type': 'Type A', 'size': 'Large', 'location': 'Los Angeles', 'max_capacity': 1603, 'current_capacity': 3402, 'safety_stock': 654}}, {'node_id': 'Supplier_5', 'node_type': 'Supplier', 'payload': {'id': 'Supplier_5', 'name': 'Name_6048', 'location': 'Chicago', 'quality': 272.75, 'reliability': 0.38, 'size': 'Extra Large'}}, {'node_id': 'Warehouse_4', 'node_type': 'Warehouse', 'payload': {'id': 'W

In [7]:
delete_cycles = 0
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)

In [8]:
# schema = json.load(open("../../../metadata/relations.json", "r"))
# generator = RandomNetworkGenerator(schema)

# custom_counts = {
#     "BusinessUnit": 1,
#     "ProductFamily": 2,
#     "ProductOffering": 3,
#     "Facility": 4,
#     "Parts": 5,
#     "Warehouse": 6,
#     "Supplier": 7,
# }

# # API_URL = "https://bug-hardy-obviously.ngrok-free.app"
# API_URL = "http://localhost:8000"

# version = "test181"

# create_ops, graph = generator.create_network(custom_counts)

# print(f"Generated {len(create_ops)} operations")

# from networkx.readwrite import json_graph
# print(json_graph.node_link_data(graph[0], edges="links"))

# import os
# import sys
# sys.path.append("../../../")
# from workers import direct_create

# direct_create(payload=json_graph.node_link_data(graph[0], edges="links"), timestamp = create_ops[0]["timestamp"], version = version)
# print("Direct create complete")

# update_cycles = 5
# for _ in range(update_cycles):
#     update_ops = generator.generate_updates(
#         node_updates=10,
#         edge_updates=10
#     )
#     print(f"Generated {len(update_ops)} update operations")

#     bulk_update_payload = {
#         "version": version,
#         "action": "bulk_update",
#         "type": "schema",
#         "timestamp": update_ops[0]["timestamp"],
#         "payload": []
#     }

#     for op in update_ops:
#         bulk_update_payload["payload"].append(op["payload"])

#     print(bulk_update_payload)

#     requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
#     time.sleep(1)